# Testes de modelo
Este notebook contém os testes que deram origem a um dos modelos para predição de qual time será o primeiro a marcar um gol, dados os nomes dos dois times. Sendo assim **o objetivo deste arquivo é apenas compreender a linha de raciocínio que culminou no modelo final, não devendo ser levado em consideração para conclusões ou para momentos de avaliação.**

## Concepções iniciais e motivações

O objetivo inicial dos modelos construídos a seguir são a predição de qual time será o primeiro a pontuar em determinada rodada.

A priori, considerou-se realizar uma análise histórica, traçando o comportamento dos times que disputarão determinada partida ao longo de disputas anteriores para, assim, determinar qual tem a maior probabilidade de marcar o primeiro gol. Tal abordagem, entretanto, tem um grave dificultador: a ausência de dados suficientes. Os times se enfrentam apenas duas vezes ao longo do Campeonato Brasileiro, o que acaba por impedir uma análise histórica com alta confiabilidade.

Sendo assim, propõe-se uma abordagem alternativa com lógica semelhante a anterior. Para tal, em vez de analisar o desempenho de determinado time A contra determinado time B ao longo de disputas entre esses dois times, será averiguado o comportamento do time A contra times parecidos com o time B e vice-versa, considerando que podemos extrapolar esses comportamentos para partidas posteriores. Desse modo, será realizada uma análise de probabilidade empírica [[1]](http://tpub.com/math2/90.htm)[[2]](https://www.geeksforgeeks.org/empirical-probability/) nos dados, permitindo a previsão de jogos futuros baseado em resultados anteriores.

Os procedimentos e testes para se chegar esse modelo estão descritos a seguir:

## Código

### Setup de dependências

Importação de bibliotecas necessárias para o funcionamento do código

In [ ]:
# Importação de bibliotecas necessárias para o funcionamento do código
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import decomposition
from sklearn.cluster import KMeans


Montagem do drive para acesso aos arquivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Abertura dos arquivos

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/new_teams.csv')
df['team_name'] = df['common_name_x']
df = df.drop(columns=['common_name_x'])

matches = pd.read_csv('/content/drive/Shareddrives/Drive IBMatch/Data/brazil-serie-a-matches-2024-to-2024-stats (5).csv', sep=";")

### Análise exploratória e pré-processamento


In [ ]:
df.info()

In [ ]:
matches = matches[['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count','home_team_goal_timings', 'away_team_goal_timings','status']]
matches.head(3)

Ajuste dos nomes dos times (faz com que os times fiquem com os mesmos nomes, evitando confusões geradas por maiúsculas/minúsculas ou por nomes oficiais/populares)

In [ ]:
def check_names(name1, name2):
    # Comparar se um nome está contido no outro (ignora maiúsculas/minúsculas)
    name1_lower = name1.lower()
    name2_lower = name2.lower()
    return name1_lower in name2_lower or name2_lower in name1_lower

# Iterar pelas linhas do segundo DataFrame e comparar com o primeiro
def adjust_names(matches_column):
  for idx2, name2 in matches[matches_column].items():
      for name1 in df['team_name']:
          if check_names(name1, name2):
              matches.at[idx2, matches_column] = name1
              break

adjust_names('home_team_name')
adjust_names('away_team_name')

Aplica PCA sobre os dados para redução de dimensionalidade e plota gráfico Scree para seleção de melhor número de componentes principais.

In [ ]:
alt = df.drop(columns=df.select_dtypes(include='object')) # cria variável alternativa para análises posteriores

# aplica método pca para comparação dos resultados
pca = decomposition.PCA(n_components = 5) # Determina quantos componentes principais serão gerados
pca_features = pca.fit_transform(alt) # Aplica o algoritmo de PCA e salva os novos dados em função dos componentes principais

# Gera gráfico scree para pca
fig = px.bar(x=range(1, len(pca.explained_variance_ratio_)+1),
             y=pca.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

Plota g´rafico bidimensional para compreender como estes estão relacionados

In [ ]:
# Plota gráfico bidimensional com os dados gerados a partir do método de PCA

# Determina os eixos X e Y
x = pca_features[:, 0]
y = pca_features[:, 1]

# Plota gráfico
chart = px.scatter(x=x, y=y,
                   title="Relação entre Componentes Principais 1 e 2",
                   labels={'x':'Componente Principal 1', 'y':'Componente Principal 2'})
chart.show()

Plota mesmo gráfico anterior, mas com um novo eixo representando os valores do componente principal 3

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(pca_features,
                    x=pca_features[:, 0],
                    y=pca_features[:, 1],
                    z=pca_features[:,2],
                    title='Relação entre componentes principais 1, 2 e 3',
                    labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'z':'Componente Principal 3'})
fig.show()

Cria matriz de gráfico de dispersão com o intuito de averiguar a relação entre pares de dados

In [ ]:
# Cria matriz comparando PCAs dois a dois
px.scatter_matrix(pca_features)

## Clusterização e testes

Como anunciado na introdução deste notebook, a comparação que será utilizada para a predição ocorrerá através da comparação do comportamento do time em disputa contra times de um mesmo grupo. Nesse contexto, será necessário um metódo que seja capaz de gerar esses grupos baseados em características relevantes. Com isso, utilizou-se do algoritmo de clusterização K-Means [[3]](https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/algo-kmeans-tech-notes.html#:~:text=O%20k%2Dmeans%20%C3%A9%20um,n%C3%BAmero%20de%20atributos%20da%20observa%C3%A7%C3%A3o), para geração dos grupos.

### Com PCA

Gera gráfico Elbow para seleção do número de clusters ideal

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init='auto')
    kmeans.fit(pca_features)
    wcss.append(kmeans.inertia_)

px.line(x=range(1, 11), y=wcss)

In [ ]:
# Aplica clusterização usando k-means na base de dados geradas com PCA
kmeans_pca = KMeans(n_clusters=3, n_init='auto')
kmeans_pca.fit(pca_features)
kmeans_pca.labels_ = kmeans_pca.labels_.astype(str)

kmeans_pca.labels_

In [ ]:
pca_features = pd.concat([pd.DataFrame(pca_features), pd.DataFrame(df['team_name'])], axis=1)
pca_features['Class'] = pd.Series(kmeans_pca.labels_, name="Class")
pca_features.head(20)

In [ ]:
# Plota gráfico bidimensional com os dados gerados a partir do método de PCA

px.scatter(pca_features, x=0, y=1, color='Class',
           title="Relação entre Componentes Principais 1 e 2",
           labels={'x':'Componente Principal 1', 'y':'Componente Principal 2'})

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(pca_features,
                    x=0,
                    y=1,
                    z=2,
                    color='Class',
                    title='Relação entre componentes principais 1, 2 e 3',
                    labels={'x':'Componente Principal 1', 'y':'Componente Principal 2', 'z':'Componente Principal 3'})
fig.show()


In [ ]:
px.scatter_matrix(pca_features, [0, 1, 2, 3, 4], color=kmeans_pca.labels_)

### Sem PCA

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init='auto')
    kmeans.fit(alt)
    wcss.append(kmeans.inertia_)

px.line(x=range(1, 11), y=wcss)

In [ ]:
kmeans_plain = KMeans(n_clusters=3, n_init='auto')
kmeans_plain.fit(alt)
kmeans_plain.labels_ = kmeans_plain.labels_.astype(str)

kmeans_plain.labels_

In [ ]:
alt = pd.concat([pd.DataFrame(alt), pd.DataFrame(df['team_name'])], axis=1)
alt['Class'] = pd.Series(kmeans_plain.labels_, name="Class")
alt.head(3)

In [ ]:
px.scatter(alt, x=alt.iloc[:, 0], y=alt.iloc[:, 1], color=alt['Class'],
           title="Relação entre Componentes Principais 1 e 2",
           labels={'x':'Componente Principal 1', 'y':'Componente Principal 2'})

In [ ]:
# Plota gráfico tridimensional dos pontos
fig = px.scatter_3d(x=alt.iloc[:, 0],
                    y=alt.iloc[:, 1],
                    z=alt.iloc[:,2],
                    color=alt['Class'],
                    title='Relação entre componentes principais 1, 2 e 3',
                    )
fig.show()

# Comparação por categoria de time

## Definição de funções

In [ ]:
def first_goals_count(team_name, opponent_team_class):
  team_first_goals_count = 0
  for i in matches.iterrows():
    min_target = 95
    min_opponent = 95
    if i[1]['home_team_name'] == team_name and opponent_team_class.isin([i[1]['away_team_name']]).any():
      if int(i[1]['home_team_goal_count']) != 0:
        try:
          min_target = int(str(i[1]['home_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_target = 95

      if int(i[1]['away_team_goal_count']) != 0:
        try:
          min_opponent = int(str(i[1]['away_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_opponent = 95

    elif i[1]['away_team_name'] == team_name and opponent_team_class.isin([i[1]['home_team_name']]).any():
      if int(i[1]['away_team_goal_count']) != 0:
        try:
          min_target = int(str(i[1]['away_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_target = 95

      if int(i[1]['home_team_goal_count']) != 0:
        try:
          min_opponent = int(str(i[1]['home_team_goal_timings']).split(',')[0].replace("'", '')[:2])
        except ValueError:
          min_opponent = 95

    if min_target < min_opponent:
      team_first_goals_count += 1

  return team_first_goals_count

def return_goal_probability(first_team, second_team, data):
  first_team_class = data.loc[data['team_name'] == first_team, 'Class']
  second_team_class = data.loc[data['team_name'] == second_team, 'Class']

  if first_team_class.empty or second_team_class.empty:
    return (0, 0)

  first_class_teams = data.loc[(data['Class'] == first_team_class.iloc[0]), 'team_name']
  second_class_teams = data.loc[(data['Class'] == second_team_class.iloc[0]), 'team_name']

  first_team_fg = first_goals_count(first_team, second_class_teams)
  second_team_fg = first_goals_count(second_team, first_class_teams)

  first_team_goal_status = first_team_fg/matches[(matches['status'] == 'complete') & (((matches['home_team_name'] == first_team) & (matches['away_team_name'].isin(second_class_teams).any())) | ((matches['away_team_name'] == first_team) & (matches['home_team_name'].isin(second_class_teams).any())))]['home_team_name'].count()
  second_team_goal_status = second_team_fg/matches[(matches['status'] == 'complete') & (((matches['home_team_name'] == second_team) & (matches['away_team_name'].isin(first_class_teams).any())) | ((matches['away_team_name'] == second_team) & (matches['home_team_name'].isin(first_class_teams).any())))]['home_team_name'].count()

  first_team_goal_probability = first_team_goal_status/(first_team_goal_status+second_team_goal_status)
  second_team_goal_probability = second_team_goal_status/(first_team_goal_status+second_team_goal_status)

  return (first_team_goal_probability, second_team_goal_probability)

first_team = 'Vasco da Gama'
second_team = 'Flamengo'
print(return_goal_probability(first_team, second_team, pca_features))

In [ ]:
def first_goal(row):
  # Função para limpar e converter os tempos dos gols para float
  def convert_goal_timings(goal_timings):
      if pd.isna(goal_timings) or not goal_timings:
          return []
      # Remove o apóstrofo e converte para float
      return [float(time.replace("'", "")) for time in goal_timings.split(',')]

  # Extrai os momentos dos gols como listas de floats, ou lista vazia se não houver gols
  home_goals = convert_goal_timings(row['home_team_goal_timings'])
  away_goals = convert_goal_timings(row['away_team_goal_timings'])

  # Combina os gols em uma lista de tuplas (time, tempo do gol)
  all_goals = [(row['home_team_name'], goal_time) for goal_time in home_goals] + \
              [(row['away_team_name'], goal_time) for goal_time in away_goals]

  # Se não houver gols, não há um primeiro a pontuar
  if not all_goals:
      return None

  # Ordena os gols pelo tempo e retorna o time que fez o primeiro gol
  first_goal = min(all_goals, key=lambda x: x[1])
  return first_goal[0]

In [ ]:
def define_previsions(target, reference):
  target['prevision'] = ''
  for idx, row in target.iterrows():
    prob1 = return_goal_probability(row['home_team_name'], row['away_team_name'], reference)[0]
    prob2 = return_goal_probability(row['home_team_name'], row['away_team_name'], reference)[1]
    if prob1 > prob2:
      target.at[idx, 'prevision'] = row['home_team_name']
    elif prob1 == prob2:
      target.at[idx, 'prevision'] = None
    else:
      target.at[idx, 'prevision'] = row['away_team_name']
  return target

In [ ]:
def give_results(target):
  for idx, row in target.iterrows():
    if row['first_goal'] == row['prevision']:
      target.at[idx, 'result'] = "Acerto"
    elif row['first_goal'] == None:
      target.at[idx, 'result'] = "Indeterminado"
    elif row['first_goal'] != row['prevision']:
      target.at[idx, 'result'] = "Erro"
  return target

## Análises com PCA

In [ ]:
partidas_pca = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name']]
temp = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name', 'away_team_goal_timings', 'home_team_goal_timings']]

partidas_pca['first_goal'] = temp.apply(first_goal, axis=1)
partidas_pca = define_previsions(partidas_pca, pca_features)
partidas_pca = give_results(partidas_pca)

partidas_pca.head(6)

In [ ]:
partidas_pca['result'].value_counts()

In [ ]:
px.histogram(partidas_pca, x='result', color='result', title='Distribuição dos resultados das previsões', labels={'result':'Resultado da previsão', 'count':'Contagem'})

In [ ]:
px.pie(partidas_pca, names='result', title='Distribuição dos resultados das previsões (Com PCA)', labels={'result':'Resultado da previsão'})

## Sem PCA

In [ ]:
partidas = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name']]
temp = matches.loc[matches['status']== 'complete', ['home_team_name', 'away_team_name', 'away_team_goal_timings', 'home_team_goal_timings']]

partidas['first_goal'] = temp.apply(first_goal, axis=1)
partidas = define_previsions(partidas, alt)
partidas = give_results(partidas)

partidas.head(6)

In [ ]:
partidas['result'].value_counts()

In [ ]:
px.histogram(partidas, x='result', color='result', title='Distribuição dos resultados das previsões', labels={'result':'Resultado da previsão', 'y':'Contagem'})

In [ ]:
px.pie(partidas, names='result', title='Distribuição dos resultados das previsões (Sem PCA)', labels={'result':'Resultado da previsão'})

Protótipo para front-end

In [ ]:
print(['Internacional','Criciúma', 'Fluminense', 'São Paulo', 'Vasco da Gama',
 'Corinthians', 'Atlético PR', 'Atlético GO', 'Cruzeiro', 'Vitória', 'Bahia',
 'Grêmio', 'Bragantino', 'Atlético Mineiro', 'Palmeiras', 'Fortaleza',
 'Juventude', 'Flamengo', 'Botafogo', 'Cuiabá'])

first_team = input('Digite o nome do primeiro time, conforme a lista anterior: ')
second_team = input('Digite o nome do segundo time, conforme a lista anterior: ')

prev = return_goal_probability(first_team, second_team, pca_features)

res = [{'Team': first_team, 'Probability': prev[0]}, {'Team': second_team, 'Probability': prev[1]}]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=['Probability'],  # Use apenas uma categoria no eixo Y para simular a barra de progresso
    x=[res[0]['Probability']],  # Probabilidade do primeiro time
    name=res[0]['Team'],
    orientation='h',
    marker=dict(color='#B488F2')  # Defina a cor da primeira barra
))

fig.add_trace(go.Bar(
    y=['Probability'],  # Mesmo eixo Y para empilhar
    x=[res[1]['Probability']],  # Probabilidade do segundo time
    name=res[1]['Team'],
    orientation='h',
    marker=dict(color='#88F298')  # Defina a cor da segunda barra
))

# Ajustes do layout
fig.update_layout(
    barmode='stack',  # Empilha as barras
    title='Comparação de Probabilidades',
    xaxis=dict(range=[0, 1]),  # Define o intervalo de 0 a 1 para representar porcentagens
    yaxis_title=None,
    xaxis_title='Probabilidade',
    showlegend=True
)

fig.show()